In [1]:
print('OK!')

OK!


In [22]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [4]:
PINECONE_API_KEY = "pcsk_4yNyFx_UucbubYGMXsHYzofwZFynEY1MYBQ4tMmHYmNxQmeYJ7pGqZJRUXTJNcP8enN31u"

In [5]:
# Extracting data
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf('data/')

In [7]:
# Creating text chunks
def text_chunking(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)
    return text_chunks

In [8]:
text_chunks = text_chunking(extracted_data)

In [9]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
#Testing the embedding model
query_result = embeddings.embed_query("Hello World!")

In [32]:
import pinecone
pc = pinecone.Pinecone("pcsk_4yNyFx_UucbubYGMXsHYzofwZFynEY1MYBQ4tMmHYmNxQmeYJ7pGqZJRUXTJNcP8enN31u")
index_name = 'llamedibot'
index = pc.Index('llamedibot')
os.environ['PINECONE_API_KEY'] = 'pcsk_4yNyFx_UucbubYGMXsHYzofwZFynEY1MYBQ4tMmHYmNxQmeYJ7pGqZJRUXTJNcP8enN31u'

In [33]:
#Here we used langchain's Pinecone
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embedding=embeddings,index_name=index_name)

In [35]:
docsearch = Pinecone.from_existing_index(index_name,embeddings)
query = 'What are allergies?'
docs = docsearch.similarity_search(query,k=3)
print('Result',docs)

Result [Document(metadata={}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depend on the tissues most sensi-\ntive to the allergen and whether the allergen spread sys-'), Document(metadata={}, page_content='reactions is triggered by harmless, everyday substances.\nThis is the condition known as allergy, and the offend-\ning substance is called an allergen. Common inhaled\nallergens include pollen, dust, and insect parts from tiny\nhouse mites. Common food allergens include nuts, fish,\nand milk.\nAllergic reactions involve a special set of cells in\nthe immune system known as mast cells. Mast cells\nserve as guards in the tissues where the body meets 

In [36]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer , just say don't know , don;t try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [37]:
prompt = PromptTemplate(template=prompt_template,
                        input_variables=['context','question'])
chain_type_kwargs = {'prompt':prompt}

In [39]:
llm = CTransformers(model=r'C:\Users\amish\.cache\huggingface\hub\models--TheBloke--Llama-2-7B-Chat-GGML\snapshots\76cd63c351ae389e1d4b91cab2cf470aab11864b',
                    model_type='llama',
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [40]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({'query':user_input})
    print("Response:",result['result'])

Response: Acne is a skin disorder in which the sebaceous glands become inflamed.
Response: The virus that causes AIDS is the human immunodeficiency virus (HIV).
Response: An allergy occurs when an individual's immune system overreacts to a harmless substance, such as pollen or dust. The immune system produces IgE antibodies, which attach to mast cells and release histamine into the bloodstream, leading to symptoms such as itching, sneezing, and difficulty breathing.
